In [ ]:
import pandas as pd 
import numpy as np                     # For mathematical calculations 
import seaborn as sns                  # For data visualization 
import matplotlib.pyplot as plt        # For plotting graphs 
import warnings
warnings.simplefilter(action='ignore')
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
train=pd.read_csv("Train.csv") 
test=pd.read_csv("Test.csv")


#dropping all unrelevant column
train.drop(['Gender','Education_Level',
       'Relationship_Status', 'Hometown','growth_rate'],1,inplace=True)
test.drop([ 'Gender','Education_Level',
       'Relationship_Status', 'Hometown','growth_rate'],1,inplace=True)
train.columns

In [ ]:
train_original=train.copy() 
test_original=test.copy()


train.head()

In [ ]:
train.describe()

In [ ]:
train.dtypes

In [ ]:
print(train.shape)
print(test.shape)
train.columns

In [ ]:
#sns.pairplot(train)

In [ ]:
train.isnull().sum()

In [ ]:
train['Age'].fillna(train['Age'].mean(), inplace=True) 
train['Time_of_service'].fillna(train['Time_of_service'].mean(), inplace=True) 
train['Work_Life_balance'].fillna(train['Work_Life_balance'].mode()[0], inplace=True)
train['Pay_Scale'].fillna(train['Pay_Scale'].mode()[0], inplace=True)
train['VAR2'].fillna(train['VAR2'].mean(), inplace=True)
train['VAR4'].fillna(train['VAR4'].mode()[0], inplace=True) 
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
test['Age'].fillna(test['Age'].mean(), inplace=True) 
test['Time_of_service'].fillna(test['Time_of_service'].mean(), inplace=True) 
test['Work_Life_balance'].fillna(test['Work_Life_balance'].mode()[0], inplace=True)
test['Pay_Scale'].fillna(test['Pay_Scale'].mode()[0], inplace=True)
test['VAR2'].fillna(test['VAR2'].mean(), inplace=True)
test['VAR4'].fillna(test['VAR4'].mode()[0], inplace=True) 
test.isnull().sum()

In [ ]:
#train['joining_age']=train['Age']-train['Time_of_service']
#test['joining_age']=test['Age']-test['Time_of_service']
#plt.hist(train['joining_age'])





In [ ]:

train=train.drop(['Employee_ID'],1)
empId = test['Employee_ID'].tolist()
test=test.drop(['Employee_ID'],1)

In [ ]:
X = train.drop('Attrition_rate',1) 

y = train.Attrition_rate


In [ ]:

X=pd.get_dummies(X,drop_first=True) 
train=pd.get_dummies(train,drop_first=True) 
test=pd.get_dummies(test,drop_first=True)


In [ ]:
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)
print(calc_vif(X))


In [ ]:
#Variance Inflation Factor. During regression analysis, VIF assesses whether factors are correlated to each other (multicollinearity), which could affect p-values and the model isn't going to be as reliable
#dropping column having high vif 

X.drop(['Age','VAR6','VAR1','VAR5'],1,inplace=True)
test.drop(['Age','VAR6','VAR1','VAR5'],1,inplace=True)

In [ ]:

from sklearn.model_selection import KFold
kf=KFold(n_splits=10, random_state=21, shuffle=True)
for train_index, test_index in kf.split(X):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
xgbmodel = xgb.XGBRegressor(max_depth=3, random_state=21,max_leaf_nodes=3,learning_rate=0.04,colsample_bytree=0.4,gamma=0.9)
xgbmodel

In [ ]:
xgbmodel.fit(x_train, y_train)
pred_y1 = xgbmodel.predict(x_test)
print(xgbmodel.score(x_test,y_test))
print((1-(np.sqrt(mean_squared_error(y_test,pred_y1))))*100)
mean_squared_error(y_test,pred_y1)

In [ ]:

pred_value=xgbmodel.predict(test)
D = {"Employee_ID": empId, "Attrition_rate":pred_value}
sub = pd.DataFrame(D)
sub.to_csv('sample_submissionround1.csv', index=False)